# Quickstart

This is a self contained guide on how to build a simple app and component spec
and launch it via two different schedulers.

## Installation

First thing we need to do is to install the TorchX python package which includes
the CLI and the library.

```sh
# install torchx with all dependencies
$ pip install torchx[dev]
```

See the [README](https://github.com/pytorch/torchx) for more
information on installation.

In [1]:
%%sh
torchx --help

usage: torchx [-h] [--log_level LOG_LEVEL] [--version]
              {describe,log,run,builtins,runopts,status,configure} ...

torchx CLI

optional arguments:
  -h, --help            show this help message and exit
  --log_level LOG_LEVEL
                        Python logging log level
  --version             show program's version number and exit

sub-commands:
  Use the following commands to run operations, e.g.: torchx run ${JOB_NAME}

  {describe,log,run,builtins,runopts,status,configure}


## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [2]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Writing my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [3]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Writing my_component.py


We can execute our component via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [4]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

local_cwd://torchx/hello_world-zvg0s9glp2dgsd


torchx 2021-11-09 02:36:10 INFO     Log files located in: /tmp/torchx_sque2sk9/torchx/hello_world-zvg0s9glp2dgsd/greeter/0
torchx 2021-11-09 02:36:10 INFO     Waiting for the app to finish...
greeter/0 Hello, your name!
torchx 2021-11-09 02:36:11 INFO     Job finished: SUCCEEDED


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [5]:
%%writefile Dockerfile

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Writing Dockerfile


Once we have the Dockerfile created we can create our docker image.

In [6]:
%%sh
docker build -t my_app:latest -f Dockerfile .


Step 1/2 : FROM ghcr.io/pytorch/torchx:0.1.0rc1
0.1.0rc1: Pulling from pytorch/torchx
4bbfd2c87b75: Pulling fs layer
d2e110be24e1: Pulling fs layer
889a7173dcfe: Pulling fs layer
6009a622672a: Pulling fs layer
143f80195431: Pulling fs layer
eccbe17c44e1: Pulling fs layer
d4c7af0d4fa7: Pulling fs layer
06b5edd6bf52: Pulling fs layer
f18d016c4ccc: Pulling fs layer
c0ad16d9fa05: Pulling fs layer
30587ba7fd6b: Pulling fs layer
909695be1d50: Pulling fs layer
f119a6d0a466: Pulling fs layer
88d87059c913: Pulling fs layer
6009a622672a: Waiting
143f80195431: Waiting
eccbe17c44e1: Waiting
d4c7af0d4fa7: Waiting
06b5edd6bf52: Waiting
f18d016c4ccc: Waiting
c0ad16d9fa05: Waiting
30587ba7fd6b: Waiting
909695be1d50: Waiting
f119a6d0a466: Waiting
88d87059c913: Waiting
889a7173dcfe: Download complete
d2e110be24e1: Verifying Checksum
d2e110be24e1: Download complete
4bbfd2c87b75: Verifying Checksum
4bbfd2c87b75: Download complete
6009a622672a: Verifying Checksum
6009a622672a: Download complete
eccbe17c44

We can then launch it on the local scheduler.

In [7]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

local_docker://torchx/hello_world-dcrrz0292bc2r


torchx 2021-11-09 02:38:40 INFO     Pulling container image: my_app:latest (this may take a while)
torchx 2021-11-09 02:38:40 WARNING  failed to pull image my_app:latest, falling back to local: 404 Client Error for http+docker://localhost/v1.41/images/create?tag=latest&fromImage=my_app: Not Found ("pull access denied for my_app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")
torchx 2021-11-09 02:38:41 INFO     Waiting for the app to finish...
greeter/0 Hello, your name!
torchx 2021-11-09 02:38:42 INFO     Job finished: SUCCEEDED


If you have a Kubernetes cluster you can use the [Kubernetes scheduler](schedulers/kubernetes.rst) to launch
this on the cluster instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```

## Builtins

TorchX also provides a number of builtin components with premade images. You can discover
them via:

In [8]:
%%sh
torchx builtins

Found 9 builtin components:
  1. metrics.tensorboard
  2. serve.torchserve
  3. dist.ddp
  4. utils.booth
  5. utils.copy
  6. utils.echo
  7. utils.python
  8. utils.sh
  9. utils.touch


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [9]:
%%sh
torchx run utils.echo --msg "Hello :)"

local_docker://torchx/echo-ft959ktx1pl97


torchx 2021-11-09 02:38:45 INFO     Pulling container image: ghcr.io/pytorch/torchx:0.1.1dev0 (this may take a while)
torchx 2021-11-09 02:40:53 INFO     Waiting for the app to finish...
torchx 2021-11-09 02:40:54 INFO     Job finished: SUCCEEDED
echo/0 Hello :)


## Next Steps

1. Checkout other features of the [torchx CLI](cli.rst)
2. Learn how to author more complex app specs by referencing [specs](specs.rst)
3. Browse through the collection of [builtin components](components/overview.rst)
4. Take a look at the [list of schedulers](schedulers.rst) supported by the runner
5. See which [ML pipeline platforms](pipelines.rst) you can run components on
6. See a [training app example](examples_apps/index.rst)